# Natural Language Processing

## Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

## Importing the dataset

In [2]:
dataset = pd.read_csv('Ethos_Dataset_Binary.csv')

## Cleaning the texts

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 998):
  review = re.sub('[^a-zA-Z]', ' ', dataset['comment'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Creating the Bag of Words model

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2700)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values
for i in range(0,998):
  if y[i] >= 0.5:
    y[i] = int(1)
  else:
    y[i] = int(0)

## Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential()

### Adding the Embedding Layer

In [8]:
cnn.add(tf.keras.layers.Embedding(3000, 32, input_length=2700))

### Adding the input layer and the pooling

In [9]:
cnn.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D())

### Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(128, activation='relu'))

### Adding the output layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
cnn.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
24/24 [==============================] - 3s 129ms/step - loss: 0.6913 - accuracy: 0.5281
Epoch 2/100
24/24 [==============================] - 3s 125ms/step - loss: 0.6883 - accuracy: 0.5548
Epoch 3/100
24/24 [==============================] - 3s 125ms/step - loss: 0.6748 - accuracy: 0.5548
Epoch 4/100
24/24 [==============================] - 3s 125ms/step - loss: 0.6288 - accuracy: 0.6324
Epoch 5/100
24/24 [==============================] - 3s 126ms/step - loss: 0.4925 - accuracy: 0.7968
Epoch 6/100
24/24 [==============================] - 3s 126ms/step - loss: 0.2304 - accuracy: 0.9238
Epoch 7/100
24/24 [==============================] - 3s 125ms/step - loss: 0.1051 - accuracy: 0.9719
Epoch 8/100
24/24 [==============================] - 3s 127ms/step - loss: 0.0414 - accuracy: 0.9920
Epoch 9/100
24/24 [==============================] - 3s 126ms/step - loss: 0.0188 - accuracy: 0.9973
Epoch 10/100
24/24 [==============================] - 3s 126ms/step - loss: 0.0099 - accura

## Making the Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = cnn.predict(X_test)
cm = confusion_matrix(y_test, y_pred.round())
print(cm)
accuracy_score(y_test, y_pred.round())

[[97 53]
 [39 61]]


0.632

In [16]:
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]
accuracy = (tp + tn)/(tp + tn + fp + fn)
precision = (tp)/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*precision*recall/(precision + recall)

In [17]:
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 Score: ",f1_score)

Accuracy:  0.632
Precision:  0.6466666666666666
Recall:  0.7132352941176471
F1 Score:  0.6783216783216783
